# 大規模言語モデル

近年、GPTをはじめとする大規模言語モデルが自然言語処理分野において顕著な成果を挙げ、文章の生成や論理推論など高度な課題においても高い性能を示している。それに伴う、大規模言語モデルを活用することでより効率的に人間の行動や社会現象に関する理論や仮説の検証と発展し、新たな可能性をもたらすと期待されている。

今まで説明した通り、文脈を考慮した単語埋め込みである文脈化単語埋め込み(contextualized word embedding)を計算するTransformerを大規模コーパスで自己教師あり学習(Masked LMやNext sentence prediction)で事前学習し、そのモデルを下流タスクのテータセットを使って微調整する方法は、自然言語処理でよく用いられる手法です。

こうした事前学習した大規模なニューラルネットワークは、大規模言語モデルのように呼ばれます。

特に、2020年以降にパラメータ数およびテキストデータをさらに大規模することで、大規模言語モデルの性能も飛躍的に向上し、人間と自然にやりとりできるような能力を身につけました。これによって、ファインチューニングは行なわずに、事前学習された大規模言語モデルをプロンプト(prompt)と呼ばれるテキストを通じて制御することで下流タスク方法も一般的になりつつあります。

ここでは、大規模言語モデルの近年(2023年まで)の進展について解説します。進展が激しい分野のため、ここでの解説に基づいて適宜最新の情報も各自調べてほしいです。

## モデルの大規模化とその効果

大規模言語モデルの開発が進むにつれて、モデルに含まれるパラメータ数が飛躍的に増加してきています。

2018年に発表されたBERTは3.4億個、2019年のGPT-2では15億個だったパラメータ数が2020年のGPT-3では1750億、そしてGPT-4はは100兆個のパラメータを持つと言われており、加速的に増加していることがわかります。

![](./Figure/LLMs_parameter.png)

![](./Figure/gpt_para.png)

こうした大規模化が行われている背景には、モデルの規模を大きくすることで性能が比例して改善していくという経験的な法則であるスケール則があります。

さらに、大規模言語モデルが一定の規模を超えると、タスクの性能が飛躍的に向上する現象も報告されています。こうした大規模化することで性能が改善し獲得される能力を創発的能力(emergent abilities)と呼ぶことがあります。

```{margin}
一方、大規模言語モデルが創発的能力を持つことに疑問視している声もあります。
```


![](./Figure/emergent.png)

## 文脈内学習

GPTのようなパラメータが非常に大きいモデルでは、学習時に入力される文章内には、様々なサブタスクが埋め込まれると考えられます。こうした文章の生成を学習することで、内包される様々な言語タスクへの処理能力の獲得が期待できます。
このアプローチはGPT-2から用いられており、文脈内学習(in-context learning)と呼ばれます。

```{margin}
詳細はこちらの[論文:Language Models are Few-Shot Learners](https://arxiv.org/abs/2005.14165)を参照してください。
```


![](./Figure/meta_learning.png)

モデルは新しいタスクをこなすための追加のトレーニングデータなしに（zero-shot learning）、または非常に少量のデータで（few-shot learning）多様なタスクを達成することが可能になります。


- Few-shot learning: 推論時に少数（10から100）のデモンストレーションを与えます
- One-shot learning: 推論時に一つのデモンストレーションを与えます
- Zero-shot learning: 推論時にデモンストレーションは与えられず、自然言語によるタスク指示のみが与えられます

![](./Figure/shot-learning.png)



このように、従来ファインチューニングが必要される多くのタスクは、モデルにプロンプト(prompt)と呼ばれるテキストを入力して後続けするテキストを予測するという形で解かせることが知られています。

- 基本的には、タスクの説明と多くのデモンストレーションを与えることでモデルの性能が向上していく傾向が見られます。

- zero, one-shotでも悪くない性能、few-shotでは一部ファインチューニングを用いたSOTAモデルに匹敵する性能を得ることが確認できました。

![](./Figure/in-context.png)


現在、一般的には、few-shot learningモデルの性能がファインチューニングによる教師あり学習モデルを超えることはまだ少ないが、ラベル付きデータの生成に費やすコストと時間を効率すると、プロンプトはファインチューニングによる教師あり学習の効率的な代替手段と考えられます。


![](./Figure/shot-compare.png)



## chain-of-thought推論


```{margin}
詳細はこちらの[論文:Chain-of-Thought Prompting Elicits Reasoning in Large Language Models](https://arxiv.org/abs/2201.11903)を参照してください。
```

大規模言語モデルが苦手とされるタスクの一つに他段階の推論が必要となるマルチステップ推論(multi-step reasoning)があります。

複数の段階の推論が必要な際に、推論過程の例示を与えるchain-of-thought推論(chain-of-thought reasoning)を用いることで性能が改善することが報告されています。

具体的言えば、chain-of-thought推論では、回答を加えて推論過程を示す例示を与えて、モデルが回答を行う際に推論過程を含めて出力テキストを生成するようにします。

![](./Figure/chains_of_thought.png)


さらに、chain-of-thought推論の推論過程を人間が与えるのではなく、推論過程の生成を促す「Let's think step by step」のような文字列をプロンプトの末尾に追加して、推論過程を生成されてから回答を抽出するzero-shot chain-of-thought reasoningも提案されています。

この方法を使うと、プロンプトを書き換えるだけで推論の性能を改善することができます。このことから、大規模言語モデルを使うあたってはプロンプトの与え方を工夫することが重要であることがわかります。

```{margin}
詳細はこちらの[論文:Large Language Models are Zero-Shot Reasoners](https://arxiv.org/abs/2205.11916)を参照してください。
```

## プロンプトエンジニアリング

プロンプトを工夫することで、性能向上を図ったり、新しいタスクを解けるようにすることをプロンプトエンジニアリング(Prompt Engineering)と呼びます。

プロンプトエンジニアリングは、プロンプトの設計と開発に限らず、LLMsとのインタラクションおよび開発に役立つ幅広いスキルと技術を含みます。これは、LLMsとインタフェースすること、ビルドすること、能力を理解することに重要なスキルであり、LLMsの安全性を向上させたり、ドメイン知識や外部ツールを使用してLLMsの機能を拡張するためにプロンプトエンジニアリングを使用できます。